<a href="https://colab.research.google.com/github/ashura1234/deeplabv3-Segmentation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
useTPU = False
if not useTPU:
    model = !nvidia-smi
    GPU_model = model[8]
    print(GPU_model)
    # if ("P100" not in GPU_model):
    #     exit()
    !pip install wandb
    import wandb

|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/"
!pip install import_ipynb
if useTPU:
  !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3


In [ ]:
import import_ipynb
import argparse

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
import os
from PIL import Image
from scipy.io import loadmat
from torch.autograd import Variable
from torchvision import transforms

#Local files
import deeplab
from pascal import VOCSegmentation
from cityscapes import Cityscapes
from indoornyu import IndoorNYU
from utils import AverageMeter, inter_and_union
from tqdm.notebook import tqdm, trange

#TPU
if useTPU:
    import torch_xla
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
    assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

importing Jupyter notebook from deeplab.ipynb
importing Jupyter notebook from pascal.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from cityscapes.ipynb
importing Jupyter notebook from indoornyu.ipynb


In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--train', action='store_true', default=False,
#                     help='training mode')
# parser.add_argument('--exp', type=str, required=True,
#                     help='name of experiment')
# parser.add_argument('--gpu', type=int, default=0,
#                     help='test time gpu device id')
# parser.add_argument('--backbone', type=str, default='resnet101',
#                     help='resnet101')
# parser.add_argument('--dataset', type=str, default='pascal',
#                     help='pascal or cityscapes')
# parser.add_argument('--groups', type=int, default=None, 
#                     help='num of groups for group normalization')
# parser.add_argument('--epochs', type=int, default=30,
#                     help='num of training epochs')
# parser.add_argument('--batch_size', type=int, default=16,
#                     help='batch size')
# parser.add_argument('--base_lr', type=float, default=0.00025,
#                     help='base learning rate')
# parser.add_argument('--last_mult', type=float, default=1.0,
#                     help='learning rate multiplier for last layers')
# parser.add_argument('--scratch', action='store_true', default=False,
#                     help='train from scratch')
# parser.add_argument('--freeze_bn', action='store_true', default=False,
#                     help='freeze batch normalization parameters')
# parser.add_argument('--weight_std', action='store_true', default=False,
#                     help='weight standardization')
# parser.add_argument('--beta', action='store_true', default=False,
#                     help='resnet101 beta')
# parser.add_argument('--crop_size', type=int, default=513,
#                     help='image crop size')
# parser.add_argument('--resume', type=str, default=None,
#                     help='path to checkpoint to resume from')
# parser.add_argument('--workers', type=int, default=4,
#                     help='number of data loading workers')
# args = parser.parse_args()
class Arguments:
    def __init__(self, 
                 exp: str,
                 fname_path:str,
                 train: bool=True,
                 pred_path: str=None,
                 color_map_path: str=None,
                 gpu: int=0,
                 backbone: str="resnet101", 
                 dataset: str="pascal", 
                 groups: int=None, 
                 epochs: int=30, 
                 batch_size: int=16, 
                 base_lr: float=0.00025, 
                 last_mult: float=1.0, 
                 scratch: bool=False, 
                 freeze_bn: bool=False,
                 weight_std: bool=False,
                 beta: bool=False,
                 crop_size: int=161,
                 resume: str=None,
                 workers: int=4
                ):
        self.exp = exp
        self.fname_path = fname_path
        self.train = train
        self.pred_path = pred_path
        self.color_map_path = color_map_path
        self.gpu = gpu
        self.backbone = backbone
        self.dataset = dataset
        self.groups=groups
        self.epochs = epochs
        self.batch_size = batch_size
        self.base_lr = base_lr
        self.last_mult = last_mult
        self.scratch = scratch
        self.freeze_bn = freeze_bn
        self.weight_std = weight_std
        self.beta = beta
        self.crop_size = crop_size
        self.resume = resume
        self.workers = workers

In [ ]:
if useTPU:
    device = xm.xla_device()
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def main():
    if not useTPU: assert torch.cuda.is_available()
    torch.backends.cudnn.benchmark = True
    model_fname = args.fname_path + 'deeplab_{0}_{1}_v3_{2}_epoch%d.pth'.format(
        args.backbone, args.dataset, args.exp)
    if args.dataset == 'pascal':
        dataset = VOCSegmentation('/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3',
        train=args.train, crop_size=args.crop_size)
    elif args.dataset == 'cityscapes':
        dataset = Cityscapes('/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/CityScapes',
        train=args.train, crop_size=args.crop_size)
    elif args.dataset == 'indoornyu':
        dataset = IndoorNYU('/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU',
        train=args.train, crop_size=args.crop_size)
    else:
        raise ValueError('Unknown dataset: {}'.format(args.dataset))
    if args.backbone == 'resnet101':
        model = getattr(deeplab, 'resnet101')(
        pretrained=(not args.scratch),
        num_classes=len(dataset.CLASSES),
        num_groups=args.groups,
        weight_std=args.weight_std,
        beta=args.beta)
    else:
        raise ValueError('Unknown backbone: {}'.format(args.backbone))

    if args.train:
        criterion = nn.CrossEntropyLoss(ignore_index=255)
        model = nn.DataParallel(model).to(device)
        model.train()
        #wandb.watch(model)
        if args.freeze_bn:
            for m in model.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()
                    m.weight.requires_grad = False
                    m.bias.requires_grad = False
        backbone_params = (
            list(model.module.conv1.parameters()) +
            list(model.module.bn1.parameters()) +
            list(model.module.layer1.parameters()) +
            list(model.module.layer2.parameters()) +
            list(model.module.layer3.parameters()) +
            list(model.module.layer4.parameters()))
        last_params = list(model.module.aspp.parameters())
        optimizer = optim.SGD([
            {'params': filter(lambda p: p.requires_grad, backbone_params)},
            {'params': filter(lambda p: p.requires_grad, last_params)}],
            lr=args.base_lr, momentum=0.9, weight_decay=0.0001)
        dataset_loader = torch.utils.data.DataLoader(
            dataset, batch_size=args.batch_size, shuffle=args.train,
            pin_memory=True, num_workers=args.workers)

        max_iter = args.epochs * len(dataset_loader)
        losses = AverageMeter()
        start_epoch = 0

        if args.resume:
            if os.path.isfile(args.resume):
                print('=> loading checkpoint {0}'.format(args.resume))
                checkpoint = torch.load(args.resume)
                start_epoch = checkpoint['epoch']
                model.load_state_dict(checkpoint['state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer'])
                print('=> loaded checkpoint {0} (epoch {1})'.format(
                  args.resume, checkpoint['epoch']))
            else:
                print('=> no checkpoint found at {0}'.format(args.resume))

        for epoch in trange(start_epoch, args.epochs):
            progress = tqdm(total=len(dataset_loader))
            for i, (inputs, target) in enumerate(dataset_loader):
                cur_iter = epoch * len(dataset_loader) + i
                lr = args.base_lr * (1 - float(cur_iter) / max_iter) ** 0.9
                optimizer.param_groups[0]['lr'] = lr
                optimizer.param_groups[1]['lr'] = lr * args.last_mult

                inputs = Variable(inputs.to(device))
                target = Variable(target.to(device))
                outputs = model(inputs)
                loss = criterion(outputs, target)
                if np.isnan(loss.item()) or np.isinf(loss.item()):
                    pdb.set_trace()
                losses.update(loss.item(), args.batch_size)

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                progress.update(1)
                progress.set_description("Current loss =  %.4f" % losses.val)
                progress.refresh() # to show immediately the update
#                 print('epoch: {0}\t'
#                       'iter: {1}/{2}\t'
#                       'lr: {3:.6f}\t'
#                       'loss: {loss.val:.4f} ({loss.ema:.4f})'.format(
#                       epoch + 1, i + 1, len(dataset_loader), lr, loss=losses))

            if epoch % 10 == 9 or epoch == args.epochs-1:
                torch.save({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    }, model_fname % (epoch + 1))
                print("Model saved at", model_fname % (epoch + 1))

        return losses
    else:
        assert pred_path
        assert color_map_path
        #torch.cuda.set_device(args.gpu)
        model = model.to(device)
        model.eval()
        checkpoint = torch.load(model_fname % args.epochs)
        state_dict = {k[7:]: v for k, v in checkpoint['state_dict'].items() if 'tracked' not in k}
        model.load_state_dict(state_dict)
        cmap = loadmat(args.color_map_path)['colormap']
        cmap = (cmap * 255).astype(np.uint8).flatten().tolist()

        inter_meter = AverageMeter()
        union_meter = AverageMeter()
        for i in range(len(dataset)):
            inputs, target = dataset[i]
            inputs = Variable(inputs.to(device))
            #print(inputs.shape)
            outputs = model(inputs.unsqueeze(0))
            _, pred = torch.max(outputs, 1)
            pred = pred.data.cpu().numpy().squeeze().astype(np.uint8)
            mask = target.numpy().astype(np.uint8)
            imname = dataset.masks[i].split('/')[-1]
            mask_pred = Image.fromarray(pred)
            mask_pred.putpalette(cmap)
            mask_pred.save(os.path.join(args.pred_path, imname))
            print('eval: {0}/{1}'.format(i + 1, len(dataset)))

            inter, union = inter_and_union(pred, mask, len(dataset.CLASSES))
            inter_meter.update(inter)
            union_meter.update(union)

        iou = inter_meter.sum / (union_meter.sum + 1e-10)
        for i, val in enumerate(iou):
              print('IoU {0}: {1:.2f}'.format(dataset.CLASSES[i], val * 100))
        print('Mean IoU: {0:.2f}'.format(iou.mean() * 100))
        return inter_meter, union_meter

In [ ]:
# path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/VOC2012/data/"
# pred_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/VOC2012/data/Predictions/"

# path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/CityScapes/data/"
# pred_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/CityScapes/data/Predictions/"

path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/"
pred_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/Predictions/"

batch_size = 128
args = Arguments(
                exp="exp bn_lr7e-3",
                dataset="indoornyu",
                fname_path=path,
                train=True,
                epochs=50,
                base_lr=0.007,
                batch_size=batch_size,
                crop_size=161,
                workers=4
                )
# if not useTPU:
#   wandb.init(
#       project='VOC2012', 
#       name='batch_size: %d' % batch_size)
#   config = wandb.config
#   config.batch_size = batch_size
losses = main()

Read 1200 files
Read 1200 files


Model saved at /content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/deeplab_resnet101_indoornyu_v3_exp bn_lr7e-3_epoch10.pth


Model saved at /content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/deeplab_resnet101_indoornyu_v3_exp bn_lr7e-3_epoch20.pth


Model saved at /content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/deeplab_resnet101_indoornyu_v3_exp bn_lr7e-3_epoch30.pth


Model saved at /content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/deeplab_resnet101_indoornyu_v3_exp bn_lr7e-3_epoch40.pth


Model saved at /content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/deeplab_resnet101_indoornyu_v3_exp bn_lr7e-3_epoch50.pth



In [ ]:
training_dataset = "indoornyu"

if training_dataset == "pascal":
    path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/VOC2012/data/"
    pred_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/VOC2012/data/Predictions/"
    color_map_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/VOC2012/data/pascal_seg_colormap.mat"
    batch_size = 16

elif training_dataset == "cityscapes":
    path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/CityScapes/data/"
    pred_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/CityScapes/data/Predictions/"
    color_map_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/CityScapes/data/pascal_seg_colormap.mat"
    batch_size = 16

elif training_dataset == "indoornyu":
    path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/"
    pred_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/Predictions/"
    color_map_path = "/content/drive/My Drive/Pytorch Projects/Image Segmentation/Deeplabv3/IndoorNYU/data/pascal_seg_colormap.mat"
    batch_size = 128

args = Arguments(
                exp="exp bn_lr7e-3",
                dataset="indoornyu",
                fname_path=path,
                pred_path=pred_path,
                color_map_path=color_map_path,
                train=False,
                epochs=50,
                batch_size=batch_size,
                workers=4
                )
inter_meter, union_meter = main()

Read 249 files
Read 249 files
eval: 1/249
eval: 2/249
eval: 3/249
eval: 4/249
eval: 5/249
eval: 6/249
eval: 7/249
eval: 8/249
eval: 9/249
eval: 10/249
eval: 11/249
eval: 12/249
eval: 13/249
eval: 14/249
eval: 15/249
eval: 16/249
eval: 17/249
eval: 18/249
eval: 19/249
eval: 20/249
eval: 21/249
eval: 22/249
eval: 23/249
eval: 24/249
eval: 25/249
eval: 26/249
eval: 27/249
eval: 28/249
eval: 29/249
eval: 30/249
eval: 31/249
eval: 32/249
eval: 33/249
eval: 34/249
eval: 35/249
eval: 36/249
eval: 37/249
eval: 38/249
eval: 39/249
eval: 40/249
eval: 41/249
eval: 42/249
eval: 43/249
eval: 44/249
eval: 45/249
eval: 46/249
eval: 47/249
eval: 48/249
eval: 49/249
eval: 50/249
eval: 51/249
eval: 52/249
eval: 53/249
eval: 54/249
eval: 55/249
eval: 56/249
eval: 57/249
eval: 58/249
eval: 59/249
eval: 60/249
eval: 61/249
eval: 62/249
eval: 63/249
eval: 64/249
eval: 65/249
eval: 66/249
eval: 67/249
eval: 68/249
eval: 69/249
eval: 70/249
eval: 71/249
eval: 72/249
eval: 73/249
eval: 74/249
eval: 75/249
eval